# Sed log maker

Basic program to produce sedimentary logs. See example input file for required format.

Prerequisites:
- Numpy
- Pandas
- drawSvg

Most Python installs will include numpy and pandas by default, but probably not drawSvg. It can be obtained from PyPi as follows:

`$ pip3 install drawSvg`

Import modules for running program. If drawings fails to import, uncomment the syspath block and make sure your present working directory is the directory this repo is cloned to. Running this cell will almost definitely throw an error about Cairo not being installed, but unless you desperately have to export your file as a PNG, you can ignore this.

In [ ]:
import numpy as np
import pandas as pd

# Leave this block commented unless the program has trouble importing 'drawings'
#import sys
#if sys.path[-1] != pwd():
#    sys.path.append(pwd())

import drawings as dr

## Input parameters

This cell contains the very basics the program needs to run, i.e., directories and specifying whether to export the final output.

In [ ]:
## Specify this stuff
filepath = 'examples/test_long.csv' # Path to your spreadsheet
out_dir = '.'                       # Path to output folder. Filenames are specified in final cell.
export = True                       # True to export file, False to not do that

## Don't change this stuff unless you really want to
na = 'NaN'
src = pd.read_csv(filepath)           # Read spreadsheet as pd.DataFrame
src = src.fillna(na)                  # Fill blank cells
elevations = dr.elevs(src.thickness)  # Convert thickness readings to elevation readings. Change the column specified if
                                      # you are using a custom input spreadsheet.

This cell contains the customisations that most people will want to make. These include:
- Vertical log scale
- Tick interval on the vertical scale
- What labels you want to apply to the log

In [ ]:
## Definitely do specify this stuff
vscale = 250           # Vertical scale of log in terms of x:1
                       # (i.e., vscale of 500 makes final log where 500 mm rock corresponds to 1 mm log)
    
tick_interval = 20     # How many metres apart do you want your ticks to be?

label = 'facies'     # Column to use as labels for units
                       # Program also accepts 'facies' or 'number' if you just want labelling to show the facies or unit number
                       # or None if you don't want any labels at all.
        
strat = 'polite'       # Labelling mode; 'polite' for labels that avoid writing on top of units,
                       # '' for labelling everything

This cell contains more fiddly changes that most people won't be interested in, like where on the page to start drawing the log, and whether you want each column of the log to be a specific height.

In [ ]:
## Probably don't bother changing this stuff
paper = (None, None, 'letter') # Array containing the paper size to be used. If you want to change this, consult
                               # help text in drawings.py to figure out how to.
origin = 40                    # X and Y distance from bottom corner of page to start log in pt
padding = 5                    # Padding around the edge of the page in pt
column_spacing = 40            # Space between columns in pt
line_weight = 0.5              # Line weight in pt
manual_column_height = None    # Specify a given column height (in m of section)
cols = None                    # Specify a number of columns

The cells below specify the lookup lists that are used to make everything the right shape and size in your log. The default lists can be found in the Readme for the project.
                            
The default canvas created by `dr.canvas` is portrait US letter. If you wish to create alternatives to better suit your application, you are referred to the `drawings.py` file and embedded help text (which I will be improving at some point). The pallette is also in the repo if you (like me, I hope) can't read hex codes.

If you're unsure, just leave default as true and create your input file accordingly.

In [ ]:
default = True

if(default is True):
    gs_codes, gs_widths = dr.grainsize()
    fa_codes, fa_colors = dr.faciesList()
    canv = dr.canvas(paper[0],paper[1],paper[2])
else:
    gs_codes = []
    gs_widths = []
    fa_codes = []
    fa_colors = []
    
    gs_codes, gs_widths = dr.grainsize(gs_codes, gs_widths, wunit='mm')
    fa_codes, fa_colors = dr.faciesList(fa_codes, fa_colors)
    canv = dr.canvas(paper[0],paper[1],paper[2])

# Draw the images
This is where the actual generation takes place. It's normally pretty snappy, but I haven't tested for more than about 500 units so your mileage may vary.

## Generate a key
The below cell generates a simple key with all the colors you have used. By default, it uses a slightly customised version of the function. If you're interested in customising the key using this program, check the help for the drawKey function in `drawings.py.`

In [ ]:
default_key = True

if(default_key is True):
    key = dr.drawKey(fa_codes, fa_colors, custom_rows = 'default')
else:
    key = dr.drawKey(fa_codes, fa_colors)

## Generate the log
The cell below draws the final log. This function is complicated and not very user friendly. I plan to improve this at some point, but for now I'd strongly recommend just working with the defaults. You are, of course, welcome to check the source code too to see what all the options actually do.

In [ ]:
%%time

log = dr.drawLog(elevations, vscale,
             src.gs_base, src.gs_top, src.code,
             gs_codes, gs_widths, fa_codes, fa_colors, canv,
             orig = origin, pad = padding, colspc = column_spacing, lnwgt = line_weight,
             man_colheight = manual_column_height, columns = cols, ticks = tick_interval,
             labels = label, label_strat = strat, nachar = na)

In [ ]:
if export is True:
    log.saveSvg(f'{out_dir}/log.svg')
    key.saveSvg(f'{out_dir}/key.svg')